In [1]:
from plot import score_metrics_and_granularity
from geogiant.common.settings import PathSettings, ClickhouseSettings

path_settings = PathSettings()
clickhouse_settings = ClickhouseSettings()

# Tier 3: granularity

In [2]:
score_metrics_and_granularity(
    eval_file=path_settings.RESULTS_PATH
    / "tier3_evaluation/results__best_hostname_geo_score.pickle",
    output_path="granularity_evaluation",
    metric_evaluated="d_error",
    score_metrics=["jaccard"],
    granularities=[
        "answers",
        "answer_subnets",
        "answer_bgp_prefixes",
    ],
    probing_budgets_evaluated=[1, 50],
    plot_zp=False,
)

2024-05-07 18:19:24.028 | INFO     | plot:score_metrics_and_granularity:593 - eval_file=PosixPath('/storage/hugo/geogiant/geogiant/common/../results/tier3_evaluation/results__best_hostname_geo_score.pickle') loaded
2024-05-07 18:19:24.287 | DEBUG    | plot:score_metrics_and_granularity:600 - ANSWERS
2024-05-07 18:19:24.292 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: jaccard, <40km=0.78, None hostnames
2024-05-07 18:19:24.294 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: jaccard, median_error=6.36 [km], None hostnames
2024-05-07 18:19:24.296 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: jaccard, <40km=0.44, None hostnames
2024-05-07 18:19:24.297 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: jaccard, median_error=74.1 [km], None hostnames
2024-05-07 18:19:24.304 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: jaccard, <40km=0.78, None hostnames
2024-05-07 18:19:24.306 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: jaccard, median

# TIER 3: similarity scores

In [5]:
# scope other graph
score_metrics_and_granularity(
    eval_file=path_settings.RESULTS_PATH
    / "tier3_evaluation/results__best_hostname_geo_score.pickle",
    output_path="score_metrics_evaluation",
    metric_evaluated="d_error",
    granularities=["answer_subnets"],
    score_metrics=[
        "jaccard",
        "intersection",
    ],
    probing_budgets_evaluated=[50, 1],
    plot_zp=False,
    legend_size=9,
)

2024-05-07 18:19:48.576 | INFO     | plot:score_metrics_and_granularity:593 - eval_file=PosixPath('/storage/hugo/geogiant/geogiant/common/../results/tier3_evaluation/results__best_hostname_geo_score.pickle') loaded
2024-05-07 18:19:48.589 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: intersection, <40km=0.77, None hostnames
2024-05-07 18:19:48.590 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: intersection, median_error=6.24 [km], None hostnames
2024-05-07 18:19:48.592 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: intersection, <40km=0.4, None hostnames
2024-05-07 18:19:48.594 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: intersection, median_error=102.86 [km], None hostnames
2024-05-07 18:19:48.598 | INFO     | plot:plot_ecs_shortest_ping:512 - ECS SP:: jaccard, <40km=0.78, None hostnames
2024-05-07 18:19:48.600 | INFO     | plot:plot_ecs_shortest_ping:515 - ECS SP:: jaccard, median_error=6.25 [km], None hostnames
2024-05-07 18:19:48.603 | INFO  

In [ ]:
# scope other graph
score_metrics_and_granularity(
    eval_file=path_settings.RESULTS_PATH
    / "tier3_evaluation/results__best_hostname_geo_score.pickle",
    output_path="score_metric_evaluation_with_scope",
    metric_evaluated="d_error",
    granularities=["answer_subnets"],
    score_metrics=[
        "jaccard_scope_linear_weight",
        "intersection_scope_linear_weight",
        "jaccard_scope_poly_weight",
        "intersection_scope_poly_weight",
    ],
    plot_zp=False,
)

2024-05-07 18:18:01.124 | INFO     | plot:score_metrics_and_granularity:591 - eval_file=PosixPath('/storage/hugo/geogiant/geogiant/common/../results/tier3_evaluation/results__best_hostname_geo_score.pickle') loaded
2024-05-07 18:18:01.139 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: intersection_scope_linear_weight, <40km=0.78, None hostnames
2024-05-07 18:18:01.141 | INFO     | plot:plot_ecs_shortest_ping:514 - ECS SP:: intersection_scope_linear_weight, median_error=6.26 [km], None hostnames
2024-05-07 18:18:01.143 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: intersection_scope_poly_weight, <40km=0.77, None hostnames
2024-05-07 18:18:01.144 | INFO     | plot:plot_ecs_shortest_ping:514 - ECS SP:: intersection_scope_poly_weight, median_error=6.46 [km], None hostnames
2024-05-07 18:18:01.146 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: jaccard_scope_linear_weight, <40km=0.78, None hostnames
2024-05-07 18:18:01.146 | INFO     | plot:plot_ecs_shortest_ping:

# TIER 3: VP selection

In [ ]:
score_metrics_and_granularity(
    eval_file=path_settings.RESULTS_PATH
    / "tier3_evaluation/results__best_hostname_geo_score.pickle",
    output_path="vp_selection_evaluation",
    metric_evaluated="d_error",
    granularities=["answer_subnets"],
    score_metrics=["jaccard"],
    probing_budgets_evaluated=[1, 10, 20, 50, 100, 500],
    plot_zp=False,
)

2024-05-07 18:18:07.257 | INFO     | plot:score_metrics_and_granularity:591 - eval_file=PosixPath('/storage/hugo/geogiant/geogiant/common/../results/tier3_evaluation/results__best_hostname_geo_score.pickle') loaded
2024-05-07 18:18:07.265 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: jaccard, <40km=0.8, None hostnames
2024-05-07 18:18:07.266 | INFO     | plot:plot_ecs_shortest_ping:514 - ECS SP:: jaccard, median_error=5.51 [km], None hostnames
2024-05-07 18:18:07.268 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: jaccard, <40km=0.8, None hostnames
2024-05-07 18:18:07.269 | INFO     | plot:plot_ecs_shortest_ping:514 - ECS SP:: jaccard, median_error=6.06 [km], None hostnames
2024-05-07 18:18:07.271 | INFO     | plot:plot_ecs_shortest_ping:511 - ECS SP:: jaccard, <40km=0.78, None hostnames
2024-05-07 18:18:07.273 | INFO     | plot:plot_ecs_shortest_ping:514 - ECS SP:: jaccard, median_error=6.25 [km], None hostnames
2024-05-07 18:18:07.274 | INFO     | plot:plot_ecs_shor